In [ ]:
# %pip install pinecone-client
# %pip install python-dotenv
# %pip install openai
# %pip install chromadb-client

# %pip install llama-index-vector-stores-chroma

In [ ]:
from openai import OpenAI, AzureOpenAI
import os
from dotenv import load_dotenv

load_dotenv(dotenv_path=".env")

In [ ]:
from pinecone import Pinecone

# pc = Pinecone(api_key=os.environ["PINECONE_API_KEY"])
# index = pc.Index("test-pdf")

In [ ]:
# from openai import OpenAI
# client = OpenAI()

# response = client.embeddings.create(
#     input="Your text string goes here",
#     model="text-embedding-3-large"
# )

# print(response.data[0].embedding)

In [ ]:
client = AzureOpenAI(
  api_key = os.getenv("AZURE_OPENAI_API_KEY"),  
  api_version = "2024-02-01",
  azure_endpoint = os.getenv("AZURE_OPENAI_ENDPOINT")
)

response = client.embeddings.create(
    input="Your text string goes here",
    model="ada_gcal"
)

print(response.data[0].embedding)

In [ ]:
def generate_embeddings(text, model="ada_gcal"): # model = "deployment_name"
    return client.embeddings.create(input = [text], model=model).data[0].embedding

# df_bills['ada_v2'] = df_bills["text"].apply(lambda x : generate_embeddings (x, model = 'text-embedding-ada-002'))

In [ ]:
from langchain_community.document_loaders import PyPDFLoader
loader = PyPDFLoader("./pdf/grokking.pdf")
pages = loader.load_and_split()

In [ ]:
print(len(pages))
print()

full_document = ""

for page in pages:
  full_document += page.page_content

print(full_document)

In [ ]:
import re

# first, extract Section Headers
pattern = r"\n\d+ [A-Z][a-zA-Z\s:]+\n"

# Find all matches
matches = re.findall(pattern, full_document)

matches.insert(0, "Abstract")

# Print all section titles
for match in matches:
    print(match.strip())
    print()

In [ ]:
pattern = r"\n\d+ [A-Z][a-zA-Z\s:]+\n"

# Split the text into sections
sections = re.split(pattern, full_document)

# number of sections should match with number of section titles (aka matches of regex pattern)
# print(sections[0])
# print(sections[2])
# print(len(sections))

# Create a dictionary to store section titles and contents
section_contents = {}

# Use zip to iterate over matches and sections simultaneously
for match, section in zip(matches, sections):
    section_title = match.strip()
    content = section.strip()
    section_contents[section_title] = content

section_contents.keys()


In [ ]:
section_contents["6 Conclusion"] #TODO: process this to ONLY the abstract content

In [ ]:
for title, content in section_contents.items():
    print(title)
    print(len(content))
    print()

In [ ]:
subsection = section_contents["3 Why Generalization Occurs: Representations and Dynamics"]
subsection

In [ ]:
# then extract subsections from sections
pattern = r"\n\d+\.\d+ [A-Z][a-zA-Z\s:]+\n"

subsections = re.findall(pattern, subsection)
print(len(subsections))
for match in subsections:
    print(match.strip())
    print()

In [ ]:
# extract references from conclusion
conclusion = section_contents["6 Conclusion"]
section_contents["6 Conclusion"] = conclusion.split("References")[0]
# conclusion.split("References")[1]

section_contents["6 Conclusion"]

In [ ]:
# run subsection splitting algo

for title, section in section_contents.items():
  pattern = r"\n\d+\.\d+ [A-Z][a-zA-Z\s:]+\n"
  subsections_headers = re.findall(pattern, section)
  
  if len(subsections_headers) > 0:
    subsection_map = {}
    subsections_content = re.split(pattern, section)

    subsections_headers.insert(0, title)
    for subsection_header, subsection in zip(subsections_headers, subsections_content):
      subsection_title = subsection_header.strip()
      content = subsection.strip()
      subsection_map[subsection_title] = content

    print(subsection_map.keys())
    section_contents[title] = subsection_map
    

In [ ]:
len(section_contents["3 Why Generalization Occurs: Representations and Dynamics"])
len(section_contents["4 Delayed Generalization: A Phase Diagram"])

In [ ]:
# use this for section title-based splitting
section_contents["4 Delayed Generalization: A Phase Diagram"]['4.1 Phase diagram of a toy model']

In [ ]:
def generate_embeddings(text, model="ada_gcal"): # model = "deployment_name"
    return client.embeddings.create(input = [text], model=model).data[0].embedding

# df_bills['ada_v2'] = df_bills["text"].apply(lambda x : generate_embeddings (x, model = 'text-embedding-ada-002'))

In [92]:
import chromadb.utils.embedding_functions as embedding_functions
azure_ef = embedding_functions.OpenAIEmbeddingFunction(
                api_key = os.getenv("AZURE_OPENAI_API_KEY"),  
                api_base=os.getenv("AZURE_OPENAI_ENDPOINT"),
                api_type="azure",
                api_version = "2024-02-01",
                model_name="ada_gcal"
            )

In [89]:
import chromadb
# Example setup of the client to connect to your chroma server
client = chromadb.HttpClient(host='localhost', port=8000)

In [98]:
# client.delete_collection(name="test_pdf") 
# Delete a collection and all associated embeddings, documents, and metadata. ⚠️ This is destructive and not reversible

In [100]:
collection = client.create_collection(
        embedding_function=azure_ef,
        name="test_pdf",
        metadata={"hnsw:space": "cosine"} # l2 is the default
    )
collection = client.get_collection(name="test_pdf", embedding_function=azure_ef)

In [101]:
# collection.peek() # returns a list of the first 10 items in the collection
# collection.count() # returns the number of items in the collection
# collection.modify(name="new_name") # Rename the collection

{'ids': [],
 'embeddings': [],
 'metadatas': [],
 'documents': [],
 'data': None,
 'uris': None}

In [103]:
# collection.add(
#     documents=["lorem ipsum...", "doc2", "doc3", ...],
#     metadatas=[{"chapter": "3", "verse": "16"}, {"chapter": "3", "verse": "5"}, {"chapter": "29", "verse": "11"}, ...],
#     ids=["id1", "id2", "id3", ...]
# )

for title, content in section_contents.items():
  print(title)
  print(content)
  print()

Abstract
Towards Understanding Grokking:
An Effective Theory of Representation Learning
Ziming Liu, Ouail Kitouni, Niklas Nolte, Eric J. Michaud, Max Tegmark, Mike Williams
Department of Physics, Institute for AI and Fundamental Interactions, MIT
{zmliu,kitouni,nnolte,ericjm,tegmark,mwill}@mit.edu
Abstract
We aim to understand grokking , a phenomenon where models generalize long after
overﬁtting their training set. We present both a microscopic analysis anchored by an
effective theory and a macroscopic analysis of phase diagrams describing learning
performance across hyperparameters. We ﬁnd that generalization originates from
structured representations whose training dynamics and dependence on training set
size can be predicted by our effective theory in a toy setting. We observe empirically
the presence of four learning phases: comprehension ,grokking ,memorization , and
confusion . We ﬁnd representation learning to occur only in a “Goldilocks zone”
(including comprehension and grokki